In [81]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable

In [82]:
# 60s timeout
timeout = 60

input_data = """
benchmark    |  genmc   |  genmc  |  hmc     |  hmc    |  cdschecker  |  cdschecker  |  wmc      |  wmc    |  xmm   |  xmm    |
reorder2/    |  1296    |  .12    |  1296    |  .10    |  503         |  0.03        |  1296     |  .06    |  1296  |  .17    |
fib_bench/   |  1764    |  .10    |  1764    |  .17    |  *           |  *           |  1764     |  .19    |  1764  |  1.59  |
szymanski/1  |  32      |  .04    |  32      |  .04    |  560         |  0.24        |  44       |  .01    |  37    |  .05    |
szymanski/2  |  2216    |  .26    |  2216    |  .41    |  *           |  *           |  6284     |  .21    |  6925  |  10.33  |
dekker-bnd/  |  55      |  .05    |  55      |  .06    |  244         |  0.16        |  55       |  .01    |  55    |  .06    |
indexer/12   |  8       |  .07    |  8       |  .06    |  8           |  0.67        |  8        |  .02    |  8     |  .06    |
indexer/13   |  64      |  .11    |  64      |  .16    |  *           |  *           |  64       |  .05    |  64    |  .11    |
indexer/14   |  512     |  .54    |  512     |  1.00   |  *           |  *           |  512      |  .31    |  512   |  .65    |
indexer/15   |  4096    |  4.54   |  4096    |  8.45   |  *           |  *           |  4096     |  2.66   |  4096  |  5.53   |
"""

# singleton/   |  4       |  .04    |  24      |  .04    |  4           |  0.00        |  24       |  .01    |  4     |  .04    |
# peterson/10  |  39      |  .04    |  40      |  .05    |  306         |  0.03        |  1152     |  .23    |  59    |  .11    |
# peterson/20  |  39      |  .04    |  40      |  .04    |  1006        |  0.13        |  3892     |  2.11   |  59    |  .06    |
# peterson/30  |  39      |  .04    |  40      |  .04    |  2106        |  0.35        |  8232     |  9.08   |  59    |  .06    |


In [83]:
def un_underscore(name: str) -> str:
    return name.replace("_", "\\_")

def parse_name(name: str) -> str:
    match un_underscore(name).split('/'):
        case [a]:
            return a
        case [a, ""]:
            return a
        case [a, number]:
            return f"{a}({number})"

def parse_result(r: str) -> str:
    match r:
        case '*':
            return '{\\fontspec{Symbola}\\symbol{"231B}}'
        case "+":
            return ''
        case r:
            return r

def parse_line(line: str) -> tuple[list[str], list[str]]:
    parts = [p.strip() for p in line.split('|') if p != ""]
    name = parse_name(parts[0])
    parts = [parse_result(p) for p in parts[1:]]
    execs = [name] + [parts[i] for i in range(0, len(parts), 2)]
    times = [name] + [parts[i] for i in range(1, len(parts), 2)]
    return execs, times

lines = [l for l in input_data.splitlines() if l != ""][1:]
tuples = [parse_line(line) for line in lines]


In [84]:

for i in range(len(tuples)):
    execs = tuples[i][0][1:]
    if execs[0] != execs[4]:
        tuples[i][0][0] = f"\\textcolor{{red}}{{{tuples[i][0][0]}}}"
        tuples[i][1][0] = tuples[i][0][0]

execs = [['Test Name', 'GenMC execs', ' HMC execs', 'CDSCheker execs', 'WMC execs', 'XMM execs']] + [exe for exe, time in tuples]
times = [['Test Name', 'GenMC time', 'HMC time', 'CDSChecker time', 'WMC time', 'XMM time']] + [time for exe, time in tuples]

print(tuples)

[(['reorder2', '1296', '1296', '503', '1296', '1296'], ['reorder2', '.12', '.10', '0.03', '.06', '.17']), (['fib\\_bench', '1764', '1764', '{\\fontspec{Symbola}\\symbol{"231B}}', '1764', '1764'], ['fib\\_bench', '.10', '.17', '{\\fontspec{Symbola}\\symbol{"231B}}', '.19', '1.59']), (['\\textcolor{red}{szymanski(1)}', '32', '32', '560', '44', '37'], ['\\textcolor{red}{szymanski(1)}', '.04', '.04', '0.24', '.01', '.05']), (['\\textcolor{red}{szymanski(2)}', '2216', '2216', '{\\fontspec{Symbola}\\symbol{"231B}}', '6284', '6925'], ['\\textcolor{red}{szymanski(2)}', '.26', '.41', '{\\fontspec{Symbola}\\symbol{"231B}}', '.21', '10.33']), (['dekker-bnd', '55', '55', '244', '55', '55'], ['dekker-bnd', '.05', '.06', '0.16', '.01', '.06']), (['indexer(12)', '8', '8', '8', '8', '8'], ['indexer(12)', '.07', '.06', '0.67', '.02', '.06']), (['indexer(13)', '64', '64', '{\\fontspec{Symbola}\\symbol{"231B}}', '64', '64'], ['indexer(13)', '.11', '.16', '{\\fontspec{Symbola}\\symbol{"231B}}', '.05', '.1

In [85]:
# rows = [['Test Name', 'GenMC execs', ' HMC execs', 'CDSCheker execs', 'WMC execs', 'XMM execs']] + [[exe[0]] + [f"{e} ({t} s)" for e, t in zip(exe[1:], time[1:])] for exe, time in tuples]

# table_1 = Texttable()
# # table_1.set_cols_width(max_len)
# table_1.set_cols_align(["l"] + ["c" for i in range(len(execs[0]) - 1)])
# table_1.set_cols_valign(["m" for i in range(len(execs[0]))])
# table_1.add_rows(rows)
# # print(table_1.draw())
# # print('\nLatextable Output:')
# print(latextable.draw_latex(table_1, position='H', caption="Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM", label="table:synthetic-benchmarks-execs"))

In [86]:
# max_len = [max([len(row[i]) for row in rows]) for i in range(len(rows[0]))]

table_1 = Texttable()
# table_1.set_cols_width(max_len)
table_1.set_cols_align(["l"] + ["c" for i in range(len(execs[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(execs[0]))])
table_1.add_rows(execs)

table_2 = Texttable()
table_2.set_precision(2)
table_2.set_cols_align(["l"] + ["c" for i in range(len(times[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(times[0]))])
table_2.add_rows(times)

print(table_1.draw())

+-------------+-------------+------------+-------------+-----------+-----------+
|  Test Name  | GenMC execs |  HMC execs |  CDSCheker  | WMC execs | XMM execs |
|             |             |            |    execs    |           |           |
+=============+=============+============+=============+===========+===========+
| reorder2    |    1296     |    1296    |     503     |   1296    |   1296    |
+-------------+-------------+------------+-------------+-----------+-----------+
|             |             |            | {\fontspec{ |           |           |
| fib\_bench  |    1764     |    1764    | Symbola}\sy |   1764    |   1764    |
|             |             |            | mbol{"231B} |           |           |
|             |             |            |      }      |           |           |
+-------------+-------------+------------+-------------+-----------+-----------+
| \textcolor{ |             |            |             |           |           |
| red}{szyman |     32      

In [87]:
print(latextable.draw_latex(table_1, position='H', caption=f"Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM. Timeout set to {timeout}s.", label="table:synthetic-benchmarks-execs"))
print(latextable.draw_latex(table_2, position='H', caption=f"Execution time (seconds) on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and GenMC-XMM. Timeout set to {timeout}s.", label="table:synthetic-benchmarks-time"))

\begin{table}[H]
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|}
			\hline
			Test Name & GenMC execs &  HMC execs & CDSCheker execs & WMC execs & XMM execs \\
			\hline
			reorder2 & 1296 & 1296 & 503 & 1296 & 1296 \\
			\hline
			fib\_bench & 1764 & 1764 & {\fontspec{Symbola}\symbol{"231B}} & 1764 & 1764 \\
			\hline
			\textcolor{red}{szymanski(1)} & 32 & 32 & 560 & 44 & 37 \\
			\hline
			\textcolor{red}{szymanski(2)} & 2216 & 2216 & {\fontspec{Symbola}\symbol{"231B}} & 6284 & 6925 \\
			\hline
			dekker-bnd & 55 & 55 & 244 & 55 & 55 \\
			\hline
			indexer(12) & 8 & 8 & 8 & 8 & 8 \\
			\hline
			indexer(13) & 64 & 64 & {\fontspec{Symbola}\symbol{"231B}} & 64 & 64 \\
			\hline
			indexer(14) & 512 & 512 & {\fontspec{Symbola}\symbol{"231B}} & 512 & 512 \\
			\hline
			indexer(15) & 4096 & 4096 & {\fontspec{Symbola}\symbol{"231B}} & 4096 & 4096 \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Number of executions on synthetic benchmarks run with GenMC, HMC, CDSChecker, WMC, and 